In [1]:
combinedwkbk23 = read.csv('tippecanoe/Combined_Workbook_2023.csv')
combinedwkbk24 = read.csv('tippecanoe/Combined_Workbook_2024.csv')
combinedexc23 = read.csv('tippecanoe/Combined_Reconciliation_2023.csv')
combinedexc24 = read.csv('tippecanoe/Combined_Reconciliation_2024.csv')
combinedrat23 = read.csv('tippecanoe/Combined_Study_2023.csv')
combinedrat24 = read.csv('tippecanoe/Combined_Study_2024.csv')

In [2]:
## Study if recorded AV in 2023 Sales Reconciliation Document is Equal to Current Year AV from 2023 Workbook 
## - Each County
sigcty = c()
for (county in unique(combinedexc23$County)) {
    ctywkbk = combinedwkbk23[combinedwkbk23$County == county,]
    if (gsub(" ", "", county) %in% c('Warrick')) {
        next
        ##Warrick 2023 Sales Reconciliation is incomplete - No AV included 
    }
    ctyexc = combinedexc23[combinedexc23$County == county,]
    excluded_indices <- match(ctyexc$ParcelNumber,ctywkbk$ParcelNumber)
    print(paste(round(((sum(ctyexc$PostedAV[!is.na(excluded_indices)] != 
                        ctywkbk$CurrentAV[excluded_indices[!is.na(excluded_indices)]])) / nrow(ctyexc))*100,2)
            ,'% of the',nrow(ctyexc),'excluded sales are not using the Current Total AV from the',county,'County Workbook.'))
    if (sum(ctyexc$PostedAV[!is.na(excluded_indices)] != 
                        ctywkbk$CurrentAV[excluded_indices[!is.na(excluded_indices)]]) > 0) {
        ex_filtered_indices <- excluded_indices[!is.na(excluded_indices)]
        excluded_filtered <- ctyexc[!is.na(excluded_indices), ]
        ex_workbook_filtered <- ctywkbk[ex_filtered_indices, ]
        ex_invalid_rows <- excluded_filtered[excluded_filtered$PostedAV != ex_workbook_filtered$CurrentAV, ]
        ## Extract rows where AV is not equal to the corresponding parcel number's Current AV in the workbook
        ex_inv_indices <- match(ex_invalid_rows$ParcelNumber,ctywkbk$ParcelNumber)
        ## Re-Match these sales by parcel number
        cat("\t",round(((sum(ex_invalid_rows$PostedAV[!is.na(ex_inv_indices)] == 
                        ctywkbk$PriorAV[ex_inv_indices[!is.na(ex_inv_indices)]]) / nrow(ex_invalid_rows))*100),2)
            ,'% of the',nrow(ex_invalid_rows),'incorrect excluded sales are using the Prior Total AV from the'
            ,county,'County Workbook.',"\n")
        ex_old_med = median(ctyexc$PostedSaleRatio,na.rm=TRUE)
        ex_new_med = median(ctyexc$SaleRatio,na.rm=TRUE)
        ## Correcting the excluded sales' AV and sale ratio based on Current Total AV
        cat("\t",'The impact of the incorrect AV calculations changes the median excluded sale ratio for',county,'County by:'
            ,round((ex_new_med - ex_old_med)/ex_old_med*100,2),'%',"\n")
        if (abs((ex_new_med - ex_old_med)/ex_old_med*100) > 30) {
            sigcty = c(sigcty,county)
        }
    } 
}
print('List of Counties where Median Sales Ratio is changed by over 30%:')
print(sigcty)

[1] "89.38 % of the 405 excluded sales are not using the Current Total AV from the Adams County Workbook."
	 51.66 % of the 362 incorrect excluded sales are using the Prior Total AV from the Adams County Workbook. 
	 The impact of the incorrect AV calculations changes the median excluded sale ratio for Adams County by: 13.8 % 
[1] "92.22 % of the 3225 excluded sales are not using the Current Total AV from the Allen County Workbook."
	 85.78 % of the 2974 incorrect excluded sales are using the Prior Total AV from the Allen County Workbook. 
	 The impact of the incorrect AV calculations changes the median excluded sale ratio for Allen County by: 31.19 % 
[1] "94.25 % of the 696 excluded sales are not using the Current Total AV from the Bartholomew County Workbook."
	 75.61 % of the 656 incorrect excluded sales are using the Prior Total AV from the Bartholomew County Workbook. 
	 The impact of the incorrect AV calculations changes the median excluded sale ratio for Bartholomew County by: 

In [3]:
## Study if recorded AV in 2023 Approved Ratio Study Document is Equal to Current Year AV from 2023 Workbook 
## - Each County
sigcty = c()
for (county in unique(combinedrat23$County)) {
    ctywkbk = combinedwkbk23[combinedwkbk23$County == county,]
    if (gsub(" ", "", county) %in% c('Crawford','Perry','Spencer','Vermillion')) {
        next
        ## Crawford 2023 Approved Ratio Study is .xls file
        ## Perry 2023 Approved Ratio Study is .xls file
        ## Spencer 2023 Approved Ratio Study is .xls file
        ## Vermillion 2023 Approved Ratio Study is .xls file
    }
    ctyrat = combinedrat23[combinedrat23$County == county,]
    ratio_indices <- match(ctyrat$ParcelNumber,ctywkbk$ParcelNumber)
    print(paste(round(((sum(ctyrat$PostedAV[!is.na(ratio_indices)] != 
        ctywkbk$CurrentAV[ratio_indices[!is.na(ratio_indices)]])) / nrow(ctyrat))*100,2)
        ,'% of the',nrow(ctyrat),'ratio study sales are not using the Current Total AV from the',county,'County Workbook.'))
    if (sum(ctyrat$PostedAV[!is.na(ratio_indices)] != 
        ctywkbk$CurrentAV[ratio_indices[!is.na(ratio_indices)]]) > 0) {
        rat_filtered_indices <- ratio_indices[!is.na(ratio_indices)]
        ratio_filtered <- ctyrat[!is.na(ratio_indices), ]
        rat_workbook_filtered <- ctywkbk[rat_filtered_indices, ]
        rat_invalid_rows <- ratio_filtered[ratio_filtered$PostedAV != rat_workbook_filtered$CurrentAV, ]
        ## Extract rows where AV is not equal to the corresponding parcel number's Current AV in the workbook
        rat_inv_indices <- match(rat_invalid_rows$ParcelNumber,ctywkbk$ParcelNumber)
        ## Re-Match these sales by parcel number
        cat("\t",round(((sum(rat_invalid_rows$PostedAV[!is.na(rat_inv_indices)] == 
                        ctywkbk$PriorAV[rat_inv_indices[!is.na(rat_inv_indices)]]) / nrow(rat_invalid_rows))*100),2)
            ,'% of the',nrow(rat_invalid_rows),'incorrect ratio study sales are using the Prior Total AV from the'
            ,county,'County Workbook.',"\n")
        rat_old_med = median(ctyrat$PostedSaleRatio,na.rm=TRUE)
        rat_new_med = median(ctyrat$SaleRatio,na.rm=TRUE)
        ## Correcting the included sales' AV and sale ratio based on Current Total AV
        cat("\t",'The impact of the incorrect AV calculations changes the median ratio study sale ratio by:'
            ,round((rat_new_med - rat_old_med)/rat_old_med*100,2),'%',"\n")
        if (abs((rat_new_med - rat_old_med)/rat_old_med*100) > 0.5) {
            sigcty = c(sigcty,county)
        }
    }
}
print('List of Counties where Median Sales Ratio is changed by over 0.5%:')
print(sigcty)

[1] "1.23 % of the 733 ratio study sales are not using the Current Total AV from the Adams County Workbook."
	 0 % of the 9 incorrect ratio study sales are using the Prior Total AV from the Adams County Workbook. 
	 The impact of the incorrect AV calculations changes the median ratio study sale ratio by: 0 % 
[1] "2.87 % of the 6299 ratio study sales are not using the Current Total AV from the Allen County Workbook."
	 6.63 % of the 181 incorrect ratio study sales are using the Prior Total AV from the Allen County Workbook. 
	 The impact of the incorrect AV calculations changes the median ratio study sale ratio by: 0.28 % 
[1] "1.45 % of the 1171 ratio study sales are not using the Current Total AV from the Bartholomew County Workbook."
	 5.88 % of the 17 incorrect ratio study sales are using the Prior Total AV from the Bartholomew County Workbook. 
	 The impact of the incorrect AV calculations changes the median ratio study sale ratio by: 0.01 % 
[1] "0.6 % of the 168 ratio study sale

In [4]:
## Study if recorded AV in 2024 Sales Reconciliation Document is Equal to Current Year AV from 2024 Workbook 
## - Each County
sigcty = c()
for (county in unique(combinedexc24$County)) {
    if(county %in% c('Kosciusko','Shelby')) {
        next
    }
    ctywkbk = combinedwkbk24[combinedwkbk24$County == county,]
    ctyexc = combinedexc24[combinedexc24$County == county,]
    excluded_indices <- match(ctyexc$ParcelNumber,ctywkbk$ParcelNumber)
    print(paste(round(((sum(ctyexc$PostedAV[!is.na(excluded_indices)] != 
                        ctywkbk$CurrentAV[excluded_indices[!is.na(excluded_indices)]])) / nrow(ctyexc))*100,2)
            ,'% of the',nrow(ctyexc),'excluded sales are not using the Current Total AV from the',county,'County Workbook.'))
    if (sum(ctyexc$PostedAV[!is.na(excluded_indices)] != 
                        ctywkbk$CurrentAV[excluded_indices[!is.na(excluded_indices)]]) > 0) {
        ex_filtered_indices <- excluded_indices[!is.na(excluded_indices)]
        excluded_filtered <- ctyexc[!is.na(excluded_indices), ]
        ex_workbook_filtered <- ctywkbk[ex_filtered_indices, ]
        ex_invalid_rows <- excluded_filtered[excluded_filtered$PostedAV != ex_workbook_filtered$CurrentAV, ]
        ## Extract rows where AV is not equal to the corresponding parcel number's Current AV in the workbook
        ex_inv_indices <- match(ex_invalid_rows$ParcelNumber,ctywkbk$ParcelNumber)
        ## Re-Match these sales by parcel number
        cat("\t",round(((sum(ex_invalid_rows$PostedAV[!is.na(ex_inv_indices)] == 
                        ctywkbk$PriorAV[ex_inv_indices[!is.na(ex_inv_indices)]]) / nrow(ex_invalid_rows))*100),2)
            ,'% of the',nrow(ex_invalid_rows),'incorrect excluded sales are using the Prior Total AV from the'
            ,county,'County Workbook.',"\n")
    }
    ex_old_med = median(ctyexc$PostedSaleRatio,na.rm=TRUE)
    ex_new_med = median(ctyexc$SaleRatio,na.rm=TRUE)
    ## Correcting the excluded sales' AV and sale ratio based on Current Total AV
    cat("\t",'The impact of the incorrect AV calculations changes the median excluded sale ratio for',county,'County by:'
            ,round((ex_new_med - ex_old_med)/ex_old_med*100,2),'%',"\n")
    if (abs((ex_new_med - ex_old_med)/ex_old_med*100) > 30) {
        sigcty = c(sigcty,county)
    }
}
print('List of Counties where Median Sales Ratio is changed by over 30%:')
print(sigcty)

[1] "90.2 % of the 490 excluded sales are not using the Current Total AV from the Adams County Workbook."
	 35.07 % of the 442 incorrect excluded sales are using the Prior Total AV from the Adams County Workbook. 
	 The impact of the incorrect AV calculations changes the median excluded sale ratio for Adams County by: 15.27 % 
[1] "93.76 % of the 2517 excluded sales are not using the Current Total AV from the Allen County Workbook."
	 78.98 % of the 2360 incorrect excluded sales are using the Prior Total AV from the Allen County Workbook. 
	 The impact of the incorrect AV calculations changes the median excluded sale ratio for Allen County by: 24.61 % 
[1] "94.65 % of the 542 excluded sales are not using the Current Total AV from the Bartholomew County Workbook."
	 80.31 % of the 513 incorrect excluded sales are using the Prior Total AV from the Bartholomew County Workbook. 
	 The impact of the incorrect AV calculations changes the median excluded sale ratio for Bartholomew County by: 

In [5]:
## Study if recorded AV in 2024 Approved Ratio Study Document is Equal to Current Year AV from 2024 Workbook 
## - Each County
sigcty = c()
for (county in unique(combinedrat24$County)) {
    ctywkbk = combinedwkbk24[combinedwkbk24$County == county,]
    if (gsub(" ", "", county) %in% c('Crawford','Spencer')) {
        next
        ## Crawford 2024 Approved Ratio Study is in .xls format
        ## Spencer 2024 Approved Ratio Study is in .xls format
    }
    ctyrat = combinedrat24[combinedrat24$County == county,]
    ratio_indices <- match(ctyrat$ParcelNumber,ctywkbk$ParcelNumber)
    print(paste(round(((sum(ctyrat$PostedAV[!is.na(ratio_indices)] != 
        ctywkbk$CurrentAV[ratio_indices[!is.na(ratio_indices)]])) / nrow(ctyrat))*100,2)
        ,'% of the',nrow(ctyrat),'ratio study sales are not using the Current Total AV from the',county,'County Workbook.'))
    if (sum(ctyrat$PostedAV[!is.na(ratio_indices)] != 
        ctywkbk$CurrentAV[ratio_indices[!is.na(ratio_indices)]]) > 0) {
        rat_filtered_indices <- ratio_indices[!is.na(ratio_indices)]
        ratio_filtered <- ctyrat[!is.na(ratio_indices), ]
        rat_workbook_filtered <- ctywkbk[rat_filtered_indices, ]
        rat_invalid_rows <- ratio_filtered[ratio_filtered$PostedAV != rat_workbook_filtered$CurrentAV, ]
        ## Extract rows where AV is not equal to the corresponding parcel number's Current AV in the workbook
        rat_inv_indices <- match(rat_invalid_rows$ParcelNumber,ctywkbk$ParcelNumber)
        ## Re-Match these sales by parcel number
        cat("\t",round(((sum(rat_invalid_rows$PostedAV[!is.na(rat_inv_indices)] == 
                        ctywkbk$PriorAV[rat_inv_indices[!is.na(rat_inv_indices)]]) / nrow(rat_invalid_rows))*100),2)
            ,'% of the',nrow(rat_invalid_rows),'incorrect ratio study sales are using the Prior Total AV from the'
            ,county,'County Workbook.',"\n")
        rat_old_med = median(ctyrat$PostedSaleRatio, na.rm=TRUE)
        rat_new_med = median(ctyrat$SaleRatio,na.rm=TRUE)
        ## Correcting the included sales' AV and sale ratio based on Current Total AV
        cat("\t",'The impact of the incorrect AV calculations changes the median ratio study sale ratio by:'
            ,round((rat_new_med - rat_old_med)/rat_old_med*100,2),'%',"\n")
        if (abs((rat_new_med - rat_old_med)/rat_old_med*100) > 0.5) {
            sigcty = c(sigcty,county)
        }
    }
    
}
print('List of Counties where Median Sales Ratio is changed by over 0.5%:')
print(sigcty)

[1] "0 % of the 476 ratio study sales are not using the Current Total AV from the Adams County Workbook."
[1] "1.74 % of the 4946 ratio study sales are not using the Current Total AV from the Allen County Workbook."
	 5.81 % of the 86 incorrect ratio study sales are using the Prior Total AV from the Allen County Workbook. 
	 The impact of the incorrect AV calculations changes the median ratio study sale ratio by: 0.09 % 
[1] "5.01 % of the 1037 ratio study sales are not using the Current Total AV from the Bartholomew County Workbook."
	 0 % of the 52 incorrect ratio study sales are using the Prior Total AV from the Bartholomew County Workbook. 
	 The impact of the incorrect AV calculations changes the median ratio study sale ratio by: -0.21 % 
[1] "1.68 % of the 119 ratio study sales are not using the Current Total AV from the Benton County Workbook."
	 0 % of the 2 incorrect ratio study sales are using the Prior Total AV from the Benton County Workbook. 
	 The impact of the incorrect 

In [6]:
ratio_indices <- match(combinedrat24$ParcelNumber,combinedwkbk24$ParcelNumber)
print(paste(round(((sum(combinedrat24$PostedAV[!is.na(ratio_indices)] != 
        combinedwkbk24$CurrentAV[ratio_indices[!is.na(ratio_indices)]])) / nrow(combinedrat24))*100,2)
        ,'% of the',nrow(combinedrat24),"2024 ratio study sales (entire state) are not using the Current Total AV from their county's Workbook."))
rat_old_med = median(combinedrat24$PostedSaleRatio, na.rm=TRUE)
rat_new_med = median(combinedrat24$SaleRatio,na.rm=TRUE)
## Correcting the included sales' AV and sale ratio based on Current Total AV
cat("\t",'The impact of the incorrect AV calculations changes the median ratio study sale ratio by:'
    ,round((rat_new_med - rat_old_med)/rat_old_med*100,2),'% for 2024 (entire state)',"\n")
ratio_indices23 <- match(combinedrat23$ParcelNumber,combinedwkbk23$ParcelNumber)
print(paste(round(((sum(combinedrat23$PostedAV[!is.na(ratio_indices23)] != 
        combinedwkbk23$CurrentAV[ratio_indices23[!is.na(ratio_indices23)]])) / nrow(combinedrat23))*100,2)
        ,'% of the',nrow(combinedrat23),"2023 ratio study sales (entire state) are not using the Current Total AV from their county's Workbook."))
rat_old_med = median(combinedrat23$PostedSaleRatio, na.rm=TRUE)
rat_new_med = median(combinedrat23$SaleRatio,na.rm=TRUE)
## Correcting the included sales' AV and sale ratio based on Current Total AV
cat("\t",'The impact of the incorrect AV calculations changes the median ratio study sale ratio by:'
    ,round((rat_new_med - rat_old_med)/rat_old_med*100,2),'% for 2023 (entire state)',"\n")
excluded_indices <- match(combinedexc24$ParcelNumber,combinedwkbk24$ParcelNumber)
print(paste(round(((sum(combinedexc24$PostedAV[!is.na(excluded_indices)] != 
                combinedwkbk24$CurrentAV[excluded_indices[!is.na(excluded_indices)]])) / nrow(combinedexc24))*100,2)
            ,'% of the',nrow(combinedexc24),"2024 excluded sales (entire state) are not using the Current Total AV from their county's Workbook."))
ex_old_med = median(combinedexc24$PostedSaleRatio,na.rm=TRUE)
ex_new_med = median(combinedexc24$SaleRatio,na.rm=TRUE)
cat("\t",'The impact of the incorrect AV calculations changes the median excluded sale ratio by:'
            ,round((ex_new_med - ex_old_med)/ex_old_med*100,2),'% for 2024 (entire state)',"\n")
excluded_indices23 <- match(combinedexc23$ParcelNumber,combinedwkbk23$ParcelNumber)
print(paste(round(((sum(combinedexc23$PostedAV[!is.na(excluded_indices23)] != 
                combinedwkbk23$CurrentAV[excluded_indices23[!is.na(excluded_indices23)]])) / nrow(combinedexc23))*100,2)
            ,'% of the',nrow(combinedexc23),"2023 excluded sales (entire state) are not using the Current Total AV from their county's Workbook."))
ex_old_med = median(combinedexc23$PostedSaleRatio,na.rm=TRUE)
ex_new_med = median(combinedexc23$SaleRatio,na.rm=TRUE)
cat("\t",'The impact of the incorrect AV calculations changes the median excluded sale ratio by:'
            ,round((ex_new_med - ex_old_med)/ex_old_med*100,2),'% for 2023 (entire state)',"\n")

[1] "1.05 % of the 51135 2024 ratio study sales (entire state) are not using the Current Total AV from their county's Workbook."
	 The impact of the incorrect AV calculations changes the median ratio study sale ratio by: 0.07 % for 2024 (entire state) 
[1] "1.03 % of the 122849 2023 ratio study sales (entire state) are not using the Current Total AV from their county's Workbook."
	 The impact of the incorrect AV calculations changes the median ratio study sale ratio by: 0.11 % for 2023 (entire state) 
[1] "83.52 % of the 39241 2024 excluded sales (entire state) are not using the Current Total AV from their county's Workbook."
	 The impact of the incorrect AV calculations changes the median excluded sale ratio by: 21.22 % for 2024 (entire state) 
[1] "69.53 % of the 90773 2023 excluded sales (entire state) are not using the Current Total AV from their county's Workbook."
	 The impact of the incorrect AV calculations changes the median excluded sale ratio by: 30.02 % for 2023 (entire sta

In [7]:
library(boot)
median_function <- function(data, indices) {
  return(median(data[indices]))
}
set.seed(123)
explanations23 = unique(combinedexc23$Explanation)
for (exp in explanations23) {
     n = nrow(combinedexc23[combinedexc23$Explanation == exp, ])
    if (n < 20) {
        next
    }
    data_subset <- combinedexc23$SaleRatio[combinedexc23$Explanation == exp]
    data_subset <- na.omit(data_subset)
    if (length(data_subset) < 20) {
        next
    }
    bootstrap_results <- boot(data_subset, statistic = median_function, R = 1000)
    ci <- boot.ci(bootstrap_results, type = "perc")
    lower_bound <- ci$percent[4]  # Lower bound
    upper_bound <- ci$percent[5]  # Upper bound
    if (lower_bound < 1.1 & upper_bound > 0.9) {
        print(paste('Median Value of Sale Ratios Excluded due to',exp,'between:',round(lower_bound,3),'and',round(upper_bound,3),'with',length(data_subset),'examples'))
    }
}


[1] "Median Value of Sale Ratios Excluded due to SOLD MORE THAN ONCE IN ONE YEAR between: 0.907 and 0.934 with 3296 examples"
[1] "Median Value of Sale Ratios Excluded due to BUSINESS TRANSACTION between: 1.081 and 1.175 with 4968 examples"
[1] "Median Value of Sale Ratios Excluded due to PRE-EXISTING RELATIONSHIP between: 1.03 and 1.112 with 2842 examples"
[1] "Median Value of Sale Ratios Excluded due to CHANGE IN USE between: 0.924 and 0.987 with 2103 examples"
[1] "Median Value of Sale Ratios Excluded due to NO REASON PROVIDED between: 0.81 and 1.024 with 190 examples"
[1] "Median Value of Sale Ratios Excluded due to INVESTMENT/RENTAL between: 0.902 and 0.941 with 7952 examples"
[1] "Median Value of Sale Ratios Excluded due to NEW CONSTRUCTION between: 0.986 and 1.155 with 791 examples"
[1] "Median Value of Sale Ratios Excluded due to NOT ENOUGH SALES FOR TRENDING between: 0.873 and 0.905 with 2085 examples"
[1] "Median Value of Sale Ratios Excluded due to VACANT LAND between: 0.985

In [8]:
library(boot)
median_function <- function(data, indices) {
  return(median(data[indices]))
}
set.seed(123)
explanations24 = unique(combinedexc24$Explanation)
for (exp in explanations24) {
     n = nrow(combinedexc24[combinedexc24$Explanation == exp, ])
    if (n < 20) {
        next
    }
    data_subset <- combinedexc24$SaleRatio[combinedexc24$Explanation == exp]
    data_subset <- na.omit(data_subset)
    if (length(data_subset) < 20) {
        next
    }
    bootstrap_results <- boot(data_subset, statistic = median_function, R = 1000)
    ci <- boot.ci(bootstrap_results, type = "perc")
    lower_bound <- ci$percent[4]  # Lower bound
    upper_bound <- ci$percent[5]  # Upper bound
    if (lower_bound < 1.1 & upper_bound > 0.9) {
        print(paste('Median Value of Sale Ratios Excluded due to',exp,'between:',round(lower_bound,3),'and',round(upper_bound,3),'with',length(data_subset),'examples'))
    }
}


[1] "Median Value of Sale Ratios Excluded due to NOT LISTED ON OPEN MARKET between: 1.056 and 1.095 with 4933 examples"
[1] "Median Value of Sale Ratios Excluded due to SOLD MORE THAN ONCE IN ONE YEAR between: 0.884 and 0.941 with 2505 examples"
[1] "Median Value of Sale Ratios Excluded due to PRE-EXISTING RELATIONSHIP between: 0.938 and 1.049 with 1579 examples"
[1] "Median Value of Sale Ratios Excluded due to DOES NOT REFLECT MARKET VALUE between: 0.858 and 0.987 with 2179 examples"
[1] "Median Value of Sale Ratios Excluded due to BUSINESS TRANSACTION between: 0.969 and 1.059 with 2143 examples"
[1] "Median Value of Sale Ratios Excluded due to SOLD TO EXEMPT ORGANIZATION between: 0.787 and 1.238 with 100 examples"
[1] "Median Value of Sale Ratios Excluded due to DUPLICATE between: 0.861 and 0.99 with 334 examples"
[1] "Median Value of Sale Ratios Excluded due to INVALID PROPERTY CLASS between: 0.703 and 0.914 with 446 examples"
[1] "Median Value of Sale Ratios Excluded due to NEW CON